In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.stats import uniform, beta, norm
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from scipy.stats import skew, kurtosis

In [2]:
def calculate_descriptive_statistics(data):
    """
    Calculate descriptive statistics for each column in the dataset.
    
    Parameters:
        data (pd.DataFrame): DataFrame with numerical data for each index.
    
    Returns:
        pd.DataFrame: DataFrame containing Mean, Std. Dev., Skew, and Kurtosis.
    """
    stats = {
        "Mean": data.mean(),
        "Std. Dev.": data.std(),
        "Skew": data.apply(skew),
        "Kurtosis": data.apply(lambda x: kurtosis(x, fisher=True))  # Fisher=True gives excess kurtosis
    }
    
    return pd.DataFrame(stats)

In [3]:
# List of yfinance-compatible tickers
tickers = [
    "SPY",      # S&P 500 ETF (large-cap U.S. equities)
    "IWM",      # iShares Russell 2000 ETF (small-cap U.S. equities)
    "QQQ",      # Nasdaq 100 ETF (tech-heavy U.S. equities)
    "IEF",      # iShares 7-10 Year Treasury Bond ETF (intermediate bonds)
    "TLT",      # iShares 20+ Year Treasury Bond ETF (long-term bonds)
    "BND",      # Vanguard Total Bond Market ETF (broad bond market)
    "VNQ",      # Vanguard Real Estate ETF (U.S. REITs)
    "GLD",      # SPDR Gold Shares (gold commodity)
    "DBC",      # Invesco DB Commodity Index Tracking Fund (broad commodities)
    "VTI"       # Vanguard Total Stock Market ETF (overall U.S. equities)
]

# Download monthly returns data for the last 14 years
start_date = "2011-11-01"
end_date = "2024-11-01"

# Fetch monthly data for each ticker
monthly_returns = {}
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date, interval='1mo', progress=False)['Adj Close']
    returns = data.pct_change().dropna() * 100  # Calculate monthly returns
    monthly_returns[ticker] = returns

# Combine all into a single DataFrame
monthly_returns_df = pd.DataFrame(monthly_returns)
monthly_returns_df.index.name = "Date"

# Abbreviation mapping for tickers
abbreviation_mapping = {
    "SPY": "USE",     # Large-cap U.S. equities
    "IWM": "USSC",    # Small-cap U.S. equities
    "QQQ": "UST",     # Technology-focused U.S. equities
    "IEF": "USB",     # Intermediate-term U.S. bonds
    "TLT": "LTB",     # Long-term U.S. bonds
    "BND": "BB",      # Broad U.S. bond market
    "VNQ": "USR",     # U.S. REITs
    "GLD": "GC",      # Gold commodity
    "DBC": "BC",      # Broad commodities
    "VTI": "TSE"      # Total U.S. equities
}

# Rename columns based on the abbreviation mapping
monthly_returns_df.rename(columns=abbreviation_mapping, inplace=True)
monthly_returns_df

,USE,USSC,UST,USB,LTB,BB,USR,GC,BC,TSE
Date,,,,,,,,,,
2011-12-01,0.408062,0.027133,-0.993085,1.812910,3.124918,0.468287,3.701074,-10.662434,-2.894349,0.405990
2012-01-01,5.301105,7.668386,8.833316,1.239534,0.238036,1.602790,7.558267,11.395481,3.688525,5.667626
2012-02-01,4.340516,2.569025,6.410073,-1.277172,-2.829983,-0.202125,-1.150715,-2.964978,5.353938,4.218476
2012-03-01,2.766024,2.171468,4.874998,-1.573870,-4.224694,-0.497547,4.361352,-1.320834,-1.773539,2.627430
2012-04-01,-0.232254,-1.316585,-1.001742,2.499252,4.801611,1.003232,3.686115,-0.148032,-1.354165,-0.210375
...,...,...,...,...,...,...,...,...,...,...
2024-06-01,3.195099,-1.399620,6.301166,1.215955,1.825939,0.877783,0.624706,-0.134700,-0.171899,2.714648
2024-07-01,1.537427,10.643647,-1.521827,2.890101,3.654331,2.353115,9.281175,5.367196,-2.798104,2.252879
2024-08-01,2.336556,-1.688551,1.103867,1.349341,2.095666,1.453755,5.220651,2.092249,-2.081485,2.131556


In [4]:
# Calculate Descriptive Statistics
descriptive_stats = calculate_descriptive_statistics(monthly_returns_df)
descriptive_stats

,Mean,Std. Dev.,Skew,Kurtosis
USE,1.223234,4.144438,-0.444680,0.978381
USSC,0.971101,5.501757,-0.333028,1.637721
UST,1.598465,5.009283,-0.273879,0.221406
USB,0.117529,1.868570,0.037403,0.112857
LTB,0.131238,3.904730,0.319341,0.167588
BB,0.152360,1.392524,-0.028800,1.141838
USR,0.800214,5.064622,-0.412085,1.282604
GC,0.351601,4.363482,0.188758,-0.083944
BC,0.039117,4.790997,-0.398904,0.449247
TSE,1.201381,4.253100,-0.459500,1.213144


In [5]:
monthly_returns_df.describe()

,USE,USSC,UST,USB,LTB,BB,USR,GC,BC,TSE
count,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000
mean,1.223234,0.971101,1.598465,0.117529,0.131238,0.152360,0.800214,0.351601,0.039117,1.201381
std,4.144438,5.501757,5.009283,1.868570,3.904730,1.392524,5.064622,4.363482,4.790997,4.253100
min,-12.998707,-21.779539,-13.488928,-4.737245,-9.442601,-4.192753,-20.016039,-11.058839,-17.340190,-14.311344
25%,-0.802316,-1.986520,-1.502416,-1.020505,-2.557421,-0.549813,-2.524572,-2.507565,-2.981861,-0.945021
50%,1.701137,1.214525,1.996827,0.031572,-0.035031,0.111989,1.127501,-0.167167,-0.038817,1.734408
75%,3.479715,4.215616,4.816792,1.218882,2.267517,0.868345,3.693594,2.960073,3.455208,3.498208
max,13.361018,18.244183,15.218749,4.634353,11.045899,4.519308,13.191199,11.395481,10.197637,13.695133
